In [1]:
import pyorc
import matplotlib.pyplot as plt
import xarray as xr
import cartopy
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from dask.diagnostics import ProgressBar
from matplotlib import patches

In [ ]:
cam_config = pyorc.load_camera_config(R"./artifact/configCam.json")
video_file = R"..\..\res\pont-cour.mp4"